In [115]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import gymnasium as gym
from dataclasses import dataclass
from torch.utils.tensorboard import SummaryWriter
import time
from IPython import display

SyntaxError: invalid syntax (306953122.py, line 10)

In [108]:
# Parmas
PERCENTILE = 70

BATCH_SIZE = 16
EPSILON = 0.5

In [110]:
model = nn.Sequential(
    nn.Linear(4, 100),
    nn.ReLU(),

    nn.Linear(100, 50),
    nn.ReLU(),

    nn.Linear(50, 1),
    nn.Sigmoid()
)

In [111]:
@dataclass
class Episode:
    observations: list[list[float]]
    actions: list[float]
    reward: float

In [112]:
env = gym.make('CartPole-v0', render_mode='rgb_array')
env.reset()

(array([-0.02835004, -0.01538559, -0.0235176 , -0.03983865], dtype=float32),
 {})

In [104]:
criterion = nn.BCELoss()
opt = optim.Adam(model.parameters(), lr=0.01)
writer = SummaryWriter()

In [114]:
epochs = 100

for epoch in range(epochs):
    episodes: list[Episode] = []

    for episode in range(BATCH_SIZE):
        episode_reward = 0
        obs, _ = env.reset()

        episode = Episode(observations=[], actions=[], reward=0)

        while True:
            obs_t = torch.tensor(obs, dtype=torch.float32).view(1, -1)
            action = int(torch.round(model(obs_t)).item())
            if np.random.rand() < EPSILON:
                action = np.random.randint(0, 2)
                
            new_obs, reward, terminated, truncated, _ = env.step(action)

            episode.observations.append(obs)
            episode.actions.append(action)
            episode.reward += reward

            episode_reward += reward
            obs = new_obs

            if terminated or truncated:
                break

            env.render()
            time.sleep(0.1)
        
        episodes.append(episode)
        
    # filter rewards
    rewards = [episode.reward for episode in episodes]
    rewards_mean = np.mean(rewards)
    percentile = np.percentile(rewards, PERCENTILE)

    writer.add_scalar('rewards_mean', rewards_mean, epoch)
    writer.add_scalar('percentile', percentile, epoch)
    writer.flush()
    
    opt.zero_grad()
    
    for episode in episodes:
        if episode.reward < percentile:
            continue

        obs_t = torch.tensor(episode.observations, dtype=torch.float32) # Bx4
        act_t = torch.tensor(episode.actions, dtype=torch.float32) # Bx1

        act_predicted = model(obs_t)
        loss = criterion(act_predicted, act_t.view(-1, 1))

        loss.backward()

    opt.step()
    
    print(f'Epoch {epoch} mean: {rewards_mean} percentile: {percentile}')

Epoch 0 mean: 13.6875 percentile: 14.5


KeyboardInterrupt: 

In [52]:
l = np.array([1,1,1,1,1,1,1,10, 20, 30])
np.percentile(l,30)

1.0